In [4]:
!pip install azure-ai-ml

In [2]:
!pip install azure-identity

In [3]:
pip install azureml-train-automl

INFO: pip is looking at multiple versions of azureml-train-automl to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of azureml-train-automl to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.2/186.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.1/107.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import pandas as pd
import os
import logging
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import automl, Input, MLClient
from pprint import pprint
from azure.ai.ml.entities import ResourceConfiguration
from sklearn.model_selection import train_test_split
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceExistsError, ResourceNotFoundError
import azureml.core
import mltable
from mltable import MLTableHeaders, MLTableFileEncoding, DataType

# <mark>**CARGA DE CREDENCIALES**</mark>

In [2]:
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

Found the config file in: /config.json


## <mark>**PROCEDER A DESCARGAR EL DATASET YA FILTRADO PARA EL ENTRENAMIENTO**</mark>

SE PROCEDIO A REALIZAR TAMBIEN LA DIVISION DEL DATA PARA LA PARTE DEL ENTRENAMIENTO EN UN 80% Y LA VALIDACION EN 20%, POSTERIORMENTE SE GUARDO LOS DATAS CREADOS PARA SU UTILIZACION EN EL ENTRENAMIENTO DEL MODELO. 

In [3]:
df = pd.read_csv('./data/bebedores_alcohol_compri.csv')
train, test = train_test_split(df, test_size=0.2, random_state=42)
train. to_csv ( './data/training-table/train_data.csv', index=False, encoding='utf-8')
test. to_csv ( './data/validation-table/valid_data.csv', index=False, encoding='utf-8')

TAMBEIN SE CREARON LOS MLTABLE DE CADA UNOS DE LOS DATASET PARA EL ENTRENAMIENTO, PARA QUE AL EL PATCH. 

In [4]:
paths = [{"file": "./data/training-table/train_data.csv"}]
tbl = mltable.from_delimited_files(
    paths= paths,
    delimiter=",",
    header=MLTableHeaders.all_files_same_headers,
    infer_column_types=True,
    include_path_column=False,
    encoding=MLTableFileEncoding.utf8,
)
mltable_folder = "./data/training-table/"
tbl.save(mltable_folder)

In [5]:
paths = [{"file": "./data/validation-table/valid_data.csv"}]
tbl = mltable.from_delimited_files(
    paths= paths,
    delimiter=",",
    header=MLTableHeaders.all_files_same_headers,
    infer_column_types=True,
    include_path_column=False,
    encoding=MLTableFileEncoding.utf8,
)
mltable_folder = "./data/validation-table/"
tbl.save(mltable_folder)

SE PRECEDE A COLOCAR LOS DATASET EN ENTRADAS UTILIZABLES PARA EL MODELO.

In [6]:
training_mltable_path = "./data/training-table/"
validation_mltable_path = "./data/validation-table/"

# Training MLTable defined locally, with local data to be uploaded
my_training_data_input = Input(type=AssetTypes.MLTABLE, path=training_mltable_path)

# Validation MLTable defined locally, with local data to be uploaded
my_validation_data_input = Input(type=AssetTypes.MLTABLE, path=validation_mltable_path)

# <mark>**CREACION DEL CLUSTER PARA LA UTILIZACION EN EL ENTRENAMIENTO DE MODELOS**</mark>

EN ESTE PASO SE ESCOGIO EL"STANDAR_E4DS_V4" YA QUE ESTE CLUSTER ESTA OPTIMIZADO PARA EL USO DE MEMORIA Y PARA DATAS NO TAN GRANDES, CONTANDO CON 4 GB DE RAM, 32GB DE MEMORIA Y UN PROCESADOR INTEL(R) XEON(R) PLATINUM 8272CL CPU @ 2.60GHZ.
SE UTILIZO EL MAXIMO CANTIDAD DE NODOS PERMITIDOS POR ESTE CLUSTER QUE SON 2.

In [42]:
compute_name = "Cpu-Cluster-E4dsv4"

try:
    _ = ml_client.compute.get(compute_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name = compute_name,
        type = "amlcompute",
        size = "Standard_E4ds_v4",
        idle_time_before_scale_down = 120,
        min_instances = 0,
        max_instances = 2,
    )
    ml_client.begin_create_or_update(compute_config).result()

Creating a new compute target...


# <mark>**Experimento**</mark>

AQUI CONFIGURAMOS EL EXPERIMENTO QUE RALIZARA UNA TAREA DE CLASIFICACION CON LOS DATAS DE ENTRENAMIENTO Y VALIDACION, CON UNA METRICA BASADA EN LA PRECISION, E UTILIZANDO LA COLUMNA DE SALIDA DE SI BEBE O NO ALCOHOL, SE PROCEDIO A TAMBIEN COLOCAR UN LIMITE DE TIEMPO QUE TRABAJE EL ENTRENAMIENTO DE 30 MINUTOS Y QUE SOLO REALICE HASTA 8 MODELOS CON UN TRABAJO EN PARALELO DE 2.

In [8]:
exp_name = 'clasificacion'
exp_timeout = 30
classification_job = automl.classification(
    compute = compute_name,
    experiment_name = exp_name,
    training_data = my_training_data_input,
    validation_data = my_validation_data_input,
    target_column_name = "DRK_YN",
    primary_metric = "accuracy",
    tags = {"my_custom_tag": "My_custon_value"},
)
classification_job.set_limits(
    timeout_minutes=exp_timeout,
    max_trials=8,
    max_concurrent_trials=2,
    )


LUEGO SE ENVIO EL JOB A AUTOML PARA QUE SE EJECUTE.

In [9]:
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

Uploading training-table (15.1 MBs): 100%|██████████| 15098852/15098852 [00:00<00:00, 56031270.46it/s]


Uploading validation-table (3.77 MBs): 100%|██████████| 3774740/3774740 [00:00<00:00, 33183420.20it/s]




Created job: compute: azureml:Cpu-Cluster-E4dsv4
creation_context:
  created_at: '2023-10-22T22:57:45.874454+00:00'
  created_by: Eddy Atahuichi
  created_by_type: User
display_name: sincere_machine_13bcjkgwpv
experiment_name: clasificacion
id: azureml:/subscriptions/ed3278d5-430f-46e8-8964-83e5db2c41c4/resourceGroups/proyecto-1-mod-1/providers/Microsoft.MachineLearningServices/workspaces/proyecto-1-now/jobs/sincere_machine_13bcjkgwpv
limits:
  enable_early_termination: true
  max_concurrent_trials: 2
  max_cores_per_trial: -1
  max_nodes: 1
  max_trials: 8
  timeout_minutes: 30
  trial_timeout_minutes: 30
log_verbosity: info
name: sincere_machine_13bcjkgwpv
outputs: {}
primary_metric: accuracy
properties: {}
resources:
  instance_count: 1
  shm_size: 2g
services:
  Studio:
    endpoint: https://ml.azure.com/runs/sincere_machine_13bcjkgwpv?wsid=/subscriptions/ed3278d5-430f-46e8-8964-83e5db2c41c4/resourcegroups/proyecto-1-mod-1/workspaces/proyecto-1-now&tid=22cb1b34-5039-43cf-a35a-8e9a0

COMANDO PARA VER EN QUE ESTADO ESTA EL ENTRENAMIENTO DE MODELOS.

In [10]:
ml_client.jobs.stream(returned_job.name) 

RunId: sincere_machine_13bcjkgwpv
Web View: https://ml.azure.com/runs/sincere_machine_13bcjkgwpv?wsid=/subscriptions/ed3278d5-430f-46e8-8964-83e5db2c41c4/resourcegroups/proyecto-1-mod-1/workspaces/proyecto-1-now

Execution Summary
RunId: sincere_machine_13bcjkgwpv
Web View: https://ml.azure.com/runs/sincere_machine_13bcjkgwpv?wsid=/subscriptions/ed3278d5-430f-46e8-8964-83e5db2c41c4/resourcegroups/proyecto-1-mod-1/workspaces/proyecto-1-now



In [11]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/ed3278d5-430f-46e8-8964-83e5db2c41c4/resourceGroups/Proyecto-1-Mod-1/providers/Microsoft.MachineLearningServices/workspaces/Proyecto-1-NOW


In [12]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [13]:
from mlflow.tracking.client import MlflowClient
from mlflow.artifacts import download_artifacts

# Initialize MLFlow client
mlflow_client = MlflowClient()

<mark>**PRECEDEMOS A OBTENER LA INFORMACION DEL JOB**</mark>

In [14]:
job = ml_client.jobs.get(name='sincere_machine_13bcjkgwpv')

In [15]:
mlflow_parent_run = mlflow_client.get_run(job.name)

In [16]:
print(mlflow_parent_run)

<Run: data=<RunData: metrics={'AUC_macro': 0.7912207111383578,
 'AUC_micro': 0.7913762520585272,
 'AUC_weighted': 0.7912207111394324,
 'accuracy': 0.7172380748231556,
 'average_precision_score_macro': 0.7859099317546936,
 'average_precision_score_micro': 0.7863915643403805,
 'average_precision_score_weighted': 0.7859148835903657,
 'balanced_accuracy': 0.7172454469474522,
 'f1_score_macro': 0.7172194282830658,
 'f1_score_micro': 0.7172380748231556,
 'f1_score_weighted': 0.7172174883479945,
 'log_loss': 0.5620810446549388,
 'matthews_correlation': 0.43455416978627653,
 'norm_macro_recall': 0.43449089389490436,
 'precision_score_macro': 0.7173087274463299,
 'precision_score_micro': 0.7172380748231556,
 'precision_score_weighted': 0.7173122209708707,
 'recall_score_macro': 0.7172454469474522,
 'recall_score_micro': 0.7172380748231556,
 'recall_score_weighted': 0.7172380748231556,
 'weighted_accuracy': 0.7172307027093824}, params={}, tags={'automl_best_child_run_id': 'sincere_machine_13bcjk

LOS MODELOS QUE UTILIZO AUTOML SON: “VOTINGENSEMBLE”, "STACKENSEMBLE”, “MAXABSSCALER, XGBOOSTCLASSIFIER”, “SPARSENORMALIZER, XGBOOSTCLASSIFIER”, “MAXABSSCALER, LIGHTGBM”, “MAXABSSCALER, EXTREMERANDOMTREES”, EL MAXABSSCALER LO UTILIZO 3 VECES PERO CON DIFERENTES HIPERPARAMETROS.
EL MEJOR MODELO QUE SACO ES EL DE VOTINGENSEBLE CON UNA PRESICION DEL 0.7167.
LOS HIPERPARAMETROS UTILIZADOS POR AUTOML SON:“algorithm : ['LightGBM', 'XGBoostClassifier', 'LightGBM', 'XGBoostClassifier']”, “min_data_in_leaf : 20”, “tree_method : auto”,  “boosting_type : gbdt”, “colsample_bytree : 0.6933333333333332”, “learning_rate : 0.09473736842105263”, “max_bin : 110, max_depth : 8, +8”, “booster : gbtree, colsample_bytree : 0.7, eta : 0.01, gamma : 0.01, max_depth : 7, +7”, ”boosting_type : gbdt, colsample_bytree : 0.5944444444444444, learning_rate : 0.06316157894736842, max_bin : 260, max_depth : 6, +8”, “bootstrap : true, class_weight : balanced, criterion : Gini, max_features : sqrt, min_samples_leaf : 0.01, +3” 


<mark>**AQUI SACAMOS EL MEJOR MODELO QUE SE ENTRENO**</mark>

In [17]:
best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]

In [18]:
best_run = mlflow_client.get_run(best_child_run_id)

In [19]:
best_run

<Run: data=<RunData: metrics={'AUC_macro': 0.7912207111383578,
 'AUC_micro': 0.7913762520585272,
 'AUC_weighted': 0.7912207111394324,
 'accuracy': 0.7172380748231556,
 'average_precision_score_macro': 0.7859099317546936,
 'average_precision_score_micro': 0.7863915643403805,
 'average_precision_score_weighted': 0.7859148835903657,
 'balanced_accuracy': 0.7172454469474522,
 'f1_score_macro': 0.7172194282830658,
 'f1_score_micro': 0.7172380748231556,
 'f1_score_weighted': 0.7172174883479945,
 'log_loss': 0.5620810446549388,
 'matthews_correlation': 0.43455416978627653,
 'norm_macro_recall': 0.43449089389490436,
 'precision_score_macro': 0.7173087274463299,
 'precision_score_micro': 0.7172380748231556,
 'precision_score_weighted': 0.7173122209708707,
 'recall_score_macro': 0.7172454469474522,
 'recall_score_micro': 0.7172380748231556,
 'recall_score_weighted': 0.7172380748231556,
 'weighted_accuracy': 0.7172307027093824}, params={}, tags={'mlflow.parentRunId': 'sincere_machine_13bcjkgwpv',

<mark>**SE PROCEDE A DESCARGAR LOS ARTEFACTOS**</mark>

In [20]:
import os

# Create local folder
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)
# Download run's artifacts/outputs
local_path = download_artifacts(
    run_id=best_run.info.run_id, artifact_path="outputs", dst_path=local_dir
)

In [21]:
# Show the contents of the MLFlow model folder
os.listdir("./artifact_downloads/outputs/mlflow-model")

['conda.yaml', 'MLmodel', 'model.pkl', 'python_env.yaml', 'requirements.txt']

<mark>**SE PROCEDE A CREAR EL ENDPOINT PARA REALIZAR EL DEPLOY**</mark>

In [22]:
# import required libraries
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
    ProbeSettings,
)

# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "clasificacion" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for deploying model",
    auth_mode="key",
    tags={"foo": "bar"},
)
print(online_endpoint_name)

clasificacion10222314412911


In [23]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://clasificacion10222314412911.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://clasificacion10222314412911.eastus.inference.ml.azure.com/swagger.json', 'name': 'clasificacion10222314412911', 'description': 'this is a sample online endpoint for deploying model', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ed3278d5-430f-46e8-8964-83e5db2c41c4/resourcegroups/proyecto-1-mod-1/providers/microsoft.machinelearningservices/workspaces/proyecto-1-now/onlineendpoints/clasificacion10222314412911', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ed3278d5-430f-46e8-8964-83e5db2c41c4/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:9ded5618-afa1-433d-88e3-a22704c1fc88:70baf5d1-9064-4a3c-b2d3-ce5332c72586?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptio

In [24]:
model_name = "classification-model"
model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/mlflow-model/",
    name=model_name,
    description="my sample classification task",
    type=AssetTypes.MLFLOW_MODEL,
)
registered_model = ml_client.models.create_or_update(model)

In [36]:
deployment = ManagedOnlineDeployment(
    name="classification-deployed",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_E4s_v3",
    instance_count=1)

ml_client.online_deployments.begin_create_or_update(deployment).result()

Check: endpoint clasificacion10222314412911 exists


..............................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'clasificacion10222314412911', 'type': 'Managed', 'name': 'classification-deployed', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ed3278d5-430f-46e8-8964-83e5db2c41c4/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/od:9ded5618-afa1-433d-88e3-a22704c1fc88:c9f2df7c-c135-4307-af06-6a0031437016?api-version=2023-04-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ed3278d5-430f-46e8-8964-83e5db2c41c4/resourceGroups/proyecto-1-mod-1/providers/Microsoft.MachineLearningServices/workspaces/proyecto-1-now/onlineEndpoints/clasificacion10222314412911/deployments/classification-deployed', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/eddyatahuichizabalaga1/code/Users/eddyatahuichizabalaga', 'creation_context': None, 'serialize':

SE UTILIZO UNA INSTANCIA STANDAD_E4S_V3 YA QUE ESTAS SI ESTAN HABILITADAS PARA REALIZAR ENDPOINT, TAMBIEN POR QUE ESTA OPTIMIZADO PARA USO DE MEMORIA.

# <mark>**PREDICCION**</mark>

In [37]:
import json

request_json = {
    "input_data": {
        "columns": ['sex', 'height', 'weight', 'waistline', 'DBP', 'triglyceride',	'hemoglobin', 'serum_creatinine',	'SGOT_ALT',	'gamma_GTP',	'SMK_stat_type_cd'],
        "data": [{'sex':0, 'height':155, 'weight':70, 'waistline':780, 'DBP':820, 'triglyceride':210, 'hemoglobin':133, 'serum_creatinine':6, 'SGOT_ALT':60, 'gamma_GTP':120, 'SMK_stat_type_cd':1}],
    }
}
request_file_name = "sample_request_data.json"
with open(request_file_name, "w") as request_file:
    json.dump(request_json, request_file)

In [89]:
resp = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=deployment.name,
    request_file=request_file_name,
)

VALORES DE 0 o 1 SI TOMO ALCOHOL.

In [90]:
print(resp)

[0]
